In [1]:
import pandas as pd
import numpy as np

# Limpieza individual de tablas

In [2]:
source_file = '../data/raw/megaline_calls.csv'
df_calls = pd.read_csv(source_file, low_memory=False)
print(f'Dimensiones antes: {df_calls.shape}')

# Convertir call_date a datetime
df_calls['call_date'] = pd.to_datetime(df_calls['call_date'], errors='coerce')
df_calls.drop('id', axis=1, inplace=True)

print(f'Dimensiones después: {df_calls.shape}')
# Guardar el archivo limpio
processed_file = '../data/processed/megaline_calls_clean.csv'
df_calls.to_csv(processed_file, index=False)

Dimensiones antes: (137735, 4)
Dimensiones después: (137735, 3)


In [3]:
source_file = '../data/raw/megaline_internet.csv'
df_internet = pd.read_csv(source_file, low_memory=False)
print(f'Dimensiones antes: {df_internet.shape}')

# Convertir session_date a datetime
df_internet['session_date'] = pd.to_datetime(df_internet['session_date'], errors='coerce')
df_internet.drop('id', axis=1, inplace=True)

print(f'Dimensiones después: {df_internet.shape}')
# Guardar el archivo limpio
processed_file = '../data/processed/megaline_internet_clean.csv'
df_internet.to_csv(processed_file, index=False)


Dimensiones antes: (104825, 4)
Dimensiones después: (104825, 3)


In [4]:
source_file = '../data/raw/megaline_messages.csv'
df_messages = pd.read_csv(source_file, low_memory=False)
print(f'Dimensiones antes: {df_messages.shape}')

# Convertir message_date a datetime
df_messages['message_date'] = pd.to_datetime(df_messages['message_date'], errors='coerce')
df_messages.drop('id', axis=1, inplace=True)

print(f'Dimensiones después: {df_messages.shape}')
# Guardar el archivo limpio
processed_file = '../data/processed/megaline_messages_clean.csv'
df_messages.to_csv(processed_file, index=False)

Dimensiones antes: (76051, 3)
Dimensiones después: (76051, 2)


In [5]:
source_file = '../data/raw/megaline_plans.csv'
df_plans = pd.read_csv(source_file, low_memory=False)
print(f'Dimensiones antes: {df_plans.shape}')

# Eliminar filas completamente duplicadas
df_plans.drop_duplicates(inplace=True)

print(f'Dimensiones después: {df_plans.shape}')
# Guardar el archivo limpio
processed_file = '../data/processed/megaline_plans_clean.csv'
df_plans.to_csv(processed_file, index=False)


Dimensiones antes: (2, 8)
Dimensiones después: (2, 8)


In [6]:
source_file = '../data/raw/megaline_users.csv'
df_users = pd.read_csv(source_file, low_memory=False)
print(f'Dimensiones antes: {df_users.shape}')

# Imputar churn_date faltante con '0000-00-00' y convertir a datetime
df_users['churn_date'] = df_users['churn_date'].fillna('2099-12-31')
df_users['churn_date'] = pd.to_datetime(df_users['churn_date'], format='%Y-%m-%d', errors='coerce')
df_users['reg_date'] = pd.to_datetime(df_users['reg_date'], format='%Y-%m-%d', errors='coerce')

# Eliminar filas completamente duplicadas
df_users.drop_duplicates(inplace=True)

print(f'Dimensiones después: {df_users.shape}')
# Guardar el archivo limpio
processed_file = '../data/processed/megaline_users_clean.csv'
df_users.to_csv(processed_file, index=False)


Dimensiones antes: (500, 8)
Dimensiones después: (500, 8)


In [7]:
# Renombrar la columna 'plan_name' a 'plan' en df_plans
df_plans.rename(columns={'plan_name': 'plan'}, inplace=True)

# Unir todas las tablas usando user_id y plan con outer para no perder registros ni generar duplicados
df_merged = df_users.merge(df_plans, on='plan', how='outer')
del df_plans, df_users

In [8]:
df_merged = df_merged.merge(df_calls, on='user_id', how='outer')
del df_calls

In [9]:
df_merged = df_merged.merge(df_messages, on='user_id', how='outer')
del df_messages

In [10]:
df_merged = df_merged.merge(df_internet, on='user_id', how='outer')
del df_internet

MemoryError: Unable to allocate 76.1 GiB for an array with shape (10212773000,) and data type int64

In [12]:
processed_file = '../data/processed/dataset_unificado.csv'
df_merged.to_csv(processed_file, index=False)

KeyboardInterrupt: 

Ya que mi computadora no soportó el join voy a adelantar parte del feature engineering para poder continuar

# Unificación en una única tabla

In [15]:
df_calls = pd.read_csv('../data/processed/megaline_calls_clean.csv', low_memory=False)

df_calls['call_date'] = pd.to_datetime(df_calls['call_date'], errors='coerce')

# Crear columna 'year_month' para agrupar por mes
df_calls['year_month'] = df_calls['call_date'].dt.to_period('M')

# Agregar estadísticas mensuales por usuario
calls_agg = df_calls.groupby(['user_id', 'year_month']).agg(
    total_calls=('duration', 'count'),
    total_duration=('duration', 'sum'),
    avg_duration=('duration', 'mean')
).reset_index()

# Guardar el agregado
agg_calls_file = '../data/processed/calls_agg.csv'
calls_agg.to_csv(agg_calls_file, index=False)

In [16]:
df_internet = pd.read_csv('../data/processed/megaline_internet_clean.csv', low_memory=False)

df_internet['session_date'] = pd.to_datetime(df_internet['session_date'], errors='coerce')

# Crear columna 'year_month'
df_internet['year_month'] = df_internet['session_date'].dt.to_period('M')

# Agregar estadísticas mensuales por usuario
internet_agg = df_internet.groupby(['user_id', 'year_month']).agg(
    total_mb=('mb_used', 'sum'),
    sessions_count=('mb_used', 'count')
).reset_index()

# Guardar el agregado
agg_internet_file = '../data/processed/internet_agg.csv'
internet_agg.to_csv(agg_internet_file, index=False)

In [17]:
df_messages = pd.read_csv('../data/processed/megaline_messages_clean.csv', low_memory=False)

df_messages['message_date'] = pd.to_datetime(df_messages['message_date'], errors='coerce')
df_messages['year_month'] = df_messages['message_date'].dt.to_period('M')

# Agregar estadísticas mensuales por usuario (se asume que cada fila es un mensaje)
messages_agg = df_messages.groupby(['user_id', 'year_month']).agg(
    total_messages=('user_id', 'count')
).reset_index()

# Guardar el agregado
agg_messages_file = '../data/processed/messages_agg.csv'
messages_agg.to_csv(agg_messages_file, index=False)

In [ ]:
# calls_agg = pd.read_csv('../data/processed/calls_agg.csv', low_memory=False)
# internet_agg = pd.read_csv('../data/processed/internet_agg.csv', low_memory=False)
# messages_agg = pd.read_csv('../data/processed/messages_agg.csv', low_memory=False)

# Convertir year_month a string para facilitar el join
calls_agg['year_month'] = calls_agg['year_month'].astype(str)
internet_agg['year_month'] = internet_agg['year_month'].astype(str)
messages_agg['year_month'] = messages_agg['year_month'].astype(str)

# Unificar las tablas de consumo con join outer para no perder información
df_consumo = calls_agg.merge(internet_agg, on=['user_id', 'year_month'], how='outer')\
                      .merge(messages_agg, on=['user_id', 'year_month'], how='outer')

# Imputar NaN en columnas de consumo con 0
cols_consumo = ['total_calls', 'total_duration', 'avg_duration', 'total_mb', 'sessions_count', 'total_messages']
df_consumo[cols_consumo] = df_consumo[cols_consumo].fillna(0)

# Cargar información de usuarios y planes
df_users = pd.read_csv('../data/processed/megaline_users_clean.csv', low_memory=False)
df_plans = pd.read_csv('../data/processed/megaline_plans_clean.csv', low_memory=False)
df_plans.columns = ['messages_included','mb_per_month_included','minutes_included','usd_monthly_pay','usd_per_gb','usd_per_message','usd_per_minute','plan']

df_users_plans = df_users.merge(df_plans, on='plan', how='left')

# Unir el consumo mensual con la información de usuarios y planes
# Cada fila representará un usuario en un mes específico
df_analitico = df_consumo.merge(df_users_plans, on='user_id', how='left')

# Guardar el dataset analítico final
final_file = '../data/processed/dataset_analitico.csv'
df_analitico.to_csv(final_file, index=False)
